In [3]:
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import os
import zarr
import s3fs
from dask.diagnostics import ProgressBar
from datetime import datetime
from dask import dataframe as dd

/tmp/ipykernel_18671/2380725633.py:10: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  from dask import dataframe as dd


In [4]:
originalFiles = '/home/ec2-user/SageMaker/datalake/notebooks/utils/data/brozen/'
outputPath = '/home/ec2-user/SageMaker/datalake/notebooks/utils/data/temporal/'

def netcdfChange(path
                 , file
                 ,outputPath
                ):
    dt = file.split('_')[-2]
    dt = datetime.strptime(dt,'%Y%m%d')
    f = xr.open_dataset(path+file)
    f = f.drop_vars(['crs'])
    f = f.rename_vars(name_dict = {'Band1':'Solar-Radiation-Flux'})
    #time_da = xr.DataArray({'time': dt})
    f = f.assign_coords(time = dt)
    f = f.expand_dims(dim="time")
    f.to_netcdf(outputPath+file)
    
    
    

In [3]:
getFile = os.listdir(originalFiles)
filespd= pd.DataFrame(getFile,columns=['file'])
filesDD = dd.from_pandas(filespd,npartitions=90)


In [4]:
with ProgressBar():
    filesDD.map_partitions(lambda df:
                          df.file.apply(lambda x: netcdfChange(originalFiles
                                                               ,x
                                                               ,outputPath)
                                       ),meta= {'name':'object'}
                          ).compute(scheduler='processes')

[########################################] | 100% Completed | 2hr 59m


In [11]:
getFile[3]

'Solar-Radiation-Flux_C3S-glob-agric_AgERA5_19870212_final-v1.0.nc'

In [12]:
netcdfChange(originalFiles
             ,'Solar-Radiation-Flux_C3S-glob-agric_AgERA5_19870212_final-v1.0.nc'
            ,outputPath)

In [5]:
s = xr.open_dataset(outputPath+'Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20230212_final-v1.0.nc')

In [8]:
s = s.to_dataframe()

In [9]:
s[s['Solar-Radiation-Flux']>0]

Solar-Radiation-Flux
time       lat        lon                              
2023-02-12 -49.975001 -75.724998             15838024.0
                      -75.674998             15807819.0
                      -75.624998             15747412.0
                      -75.574998             15259058.0
                      -75.524998             14342808.0
...                                                 ...
            49.975000  156.025005             8008318.5
                       156.075005             8023380.0
                       156.125005             8033751.5
                       156.175005             8039436.0
                       156.225005             8042277.5

[4025360 rows x 1 columns]